In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------
# 1. Define portfolio exposures
# ------------------------------
# (These are notional DV01/CS01-style sensitivities or FX position P/L per unit move)
exposures = pd.Series({
    "InterestRate_5Y": -2_000_000,   # P/L ≈ -DV01 * ΔRate
    "CreditSpread":    -1_000_000,   # P/L ≈ -CS01 * ΔSpread
    "FX_EURUSD":        5_000_000    # Long EUR vs USD (P/L ≈ Position * ΔFX)
})

factors = exposures.index

# ------------------------------
# 2. Factor volatilities (daily)
# ------------------------------
vols = pd.Series({
    "InterestRate_5Y": 0.0005,   # 5 bp
    "CreditSpread":    0.0005,   # 5 bp
    "FX_EURUSD":       0.0100    # 1 %
})

# ------------------------------
# 3. Correlation matrix
# ------------------------------
corr = pd.DataFrame(
    [[1.0,  0.30, -0.10],
     [0.30, 1.0,  -0.20],
     [-0.10, -0.20, 1.0]],
    index=factors, columns=factors)

# Covariance matrix Σ = D * Corr * D
D = np.diag(vols)
cov = D @ corr.values @ D

# ------------------------------
# 4. Monte-Carlo scenario generation
# ------------------------------
n_paths = 100_000
rng = np.random.default_rng(42)
shocks = rng.multivariate_normal(mean=np.zeros(len(factors)), cov=cov, size=n_paths)

# ------------------------------
# 5. Re-value portfolio under each scenario
# ------------------------------
pnl = shocks @ exposures.values   # shape (n_paths,)

# ------------------------------
# 6. Compute VaR & Expected Shortfall (ES)
# ------------------------------
alpha = 0.95
var_95  = -np.percentile(pnl, 100*(1-alpha))
es_95   = -pnl[pnl <= -var_95].mean()

print(f"Portfolio 1-Day 95% VaR  : {var_95:,.0f}")
print(f"Portfolio 1-Day 95% ES   : {es_95:,.0f}")

# ------------------------------
# 7. Visualise P/L distribution
# ------------------------------
plt.figure(figsize=(8,4))
plt.hist(pnl, bins=120, density=True, alpha=0.6)
plt.axvline(-var_95, linestyle="--", linewidth=2, label=f"95% VaR = {var_95:,.0f}")
plt.title("Simulated 1-Day P/L Distribution (Monte-Carlo VaR)")
plt.xlabel("P/L (USD)")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True, linestyle=":")
plt.tight_layout()
plt.show()
